In [5]:
#!/usr/bin/env python3
# -*- coding:utf-8 -*-

import enum

# https://flatmates.com.au/granny-flats+share-houses+student-accommodation+studios+whole-properties+1-beds/sydney/males+private-room?page=1


# https://flatmates.com.au/whole-properties+1-beds+homestays/sydney/males+private-room?search_source=search_function


@enum.unique
class ExtraOptions(str, enum.Enum):
    BILLS_INCLUDED = "bills-included"
    PRIVATE_ROOM = "private-room"
    SHARED_ROOM = "shared-room"
    ALLOW_MALES = "males"
    FURNISHED = "furnished"
    UNFURNISHED = "unfurnished"
    OWN = "ensuite-or-own"
    ENSUITE = "ensuite"
    MIN_AMOUNT = "min-{amount}"
    MAX_AMOUNT = "max-{amount}"


@enum.unique
class AccomodationFilter(str, enum.Enum):
    """Accommodation type filter."""

    DEFAULT = "rooms"
    PROPERTY = "whole-properties"
    SINGLE_FLAT = "1-beds"
    HOMESTAY = "homestays"
    GRANNY_FLAT = "granny-flats"
    SHARE_HOUSE = "share-houses"
    STUDENT_ACCOMMODATION = "student-accommodation"
    STUDIO = "studios"


@enum.unique
class LocationFilter(str, enum.Enum):
    """Accommodation location filter."""

    MELBOURNE = "melbourne"
    SYDNEY = "sydney"
    BRISBANE = "brisbane"
    PERTH = "perth"
    CANBERRA = "canberra"
    ADELAIDE = "adelaide"
    HOBART = "hobart"
    GOLD_COAST = "gold-coast"
    SUNSHINE_COAST = "sunshine-coast"


import bs4
import requests

# x = "https://flatmates.com.au/whole-properties+1-beds+homestays/sydney/?search_source=search_function?page=1"

# z = requests.get(x).text

# print(bs4.BeautifulSoup(z, "html.parser").select("styles__resultsWrapper___3AJFd"))


In [4]:
soup = bs4.BeautifulSoup(z, "html.parser")

In [ ]:
soup.select("a[aria-label]")

In [6]:
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup

In [ ]:
# https://flatmates.com.au/granny-flats+share-houses+student-accommodation+studios+whole-properties+1-beds/sydney/males+private-room?page=1
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup

baseurl: str = "https://flatmates.com.au/"
options: str = "/{housing_selection}/{location}/{options_selection}?page={page_number}"

for i in range(1, 230):
    opts_url: str = options.format(
        housing_selection="share-houses",
        location="sydney",
        options_selection="males+private-room",
        page_number=i,
    )

    url: str = urljoin(baseurl, opts_url)
    print(url)
    # print(requests.get(url).cookies)
    cookies = requests.get(url).cookies
    print(requests.get(url, cookies=cookies).headers)
    break
    # print(requests.get(urljoin(baseurl, opts_url)).text)
    # break
    soup = BeautifulSoup(
        requests.get("https://flatmates.com.au/sydney/males+private-room?page=1").text,
        "html.parser",
    )
    print(soup)
    # print(i, len(soup.select("a[aria-label]")))
    break

In [ ]:
baseurl: str = "https://flatmates.com.au/"
options: str = "/{housing_selection}/{location}/{options_selection}?page={page_number}"

opts_url: str = options.format(
    housing_selection="share-houses",
    location="sydney",
    options_selection="private-room",
    page_number=1,
)
url: str = urljoin(baseurl, opts_url)
print(url)
result = requests.post(url)

In [ ]:
x

In [ ]:
import requests

x = requests.get("https://flatmates.com.au/share-houses/sydney/private-room?page=1")

In [8]:
soup = BeautifulSoup(x.content, "html.parser")
# soup.select("a[aria-label]")
listings = soup.find_all("a", {"class": "styles__contentBox___37_w9"})


In [115]:
from typing import Any
from geopy.geocoders import Nominatim


class GeoLocator:
    _USER_AGENT: str = "FlatmatesApiSearch"

    def __init__(self):
        self._locator: Nominatim = Nominatim(user_agent=self._USER_AGENT)

    def locate(self, location: str) -> tuple[str, float, float]:
        pos: Any = self._locator.geocode(location)
        return pos.address, pos.latitude, pos.longitude

In [116]:
from datetime import datetime

import bs4
import pandas as pd

locator: GeoLocator = GeoLocator()


def parse_listing_tile_address(tag: bs4.Tag) -> dict[str, Any]:
    data: list[str] = tag.find("p", {"class": "styles__address___28Scu"}).contents
    address, latitude, longitude = locator.locate(data[0])
    return {"address": address, "latitude": latitude, "longitude": longitude}


def parse_listing_tile_price(tag: bs4.Tag) -> dict[str, Any]:
    data: list[str] = tag.find("p", {"class": "styles__price___3Jhqs"}).contents
    return {
        "price": float(data[0].lstrip("$").split("-")[0]),
        "paid_weekly": any(["/ week" in c for c in data]),
        "bills_included": any(["inc. bills" in c for c in data]),
    }


def parse_listing_tile_availability(tag: bs4.Tag) -> dict[str, Any]:
    data: list[str] = tag.find("p", {"class": "styles__availability___UzGsZ"}).contents
    time: datetime = datetime.now()
    if (time_str := data[0].lower().lstrip("available ")) != "now":
        time = datetime.strptime(time_str, "%d %B %Y")
    return {"available": time.date()}


def parse_listing_tile_amenities(tag: bs4.Tag) -> dict[str, Any]:
    data: list = tag.find_all("div", {"class": "styles__propertyFeature___uH480"})
    return {"bedrooms": data[0].text, "bathrooms": data[1].text, "people": data[2].text}


def parse_listing_tile_extra_info(tag: bs4.Tag) -> dict[str, Any]:
    data: list = tag.find("p", {"class": "styles__roomInfo___1BEdy"})
    return {"extra_info": data.contents[0]}


def parse_listing_tile_link(tag: bs4.Tag) -> dict[str, Any]:
    return {"link": urljoin("https://flatmates.com.au", tag.get("href"))}


def parse_listing_tile(tile: bs4.Tag) -> dict[str, Any]:
    results: dict[str, Any] = {}
    results |= parse_listing_tile_address(tile)
    results |= parse_listing_tile_price(tile)
    results |= parse_listing_tile_availability(tile)
    results |= parse_listing_tile_amenities(tile)
    results |= parse_listing_tile_extra_info(tile)
    results |= parse_listing_tile_link(tile)
    return results


In [ ]:
import pathlib


def download_flatmates_listings(
    baseurl: str,
    csv_path: pathlib.Path,
    max_page: int = 250,
) -> pd.DataFrame:
    df: pd.DataFrame = pd.DataFrame()
    if csv_path.exists() and csv_path.is_file():
        df = pd.read_csv(index_col=None)


baseurl: str = "https://flatmates.com.au/rooms/sydney/males+private-room?page={page}"
csv_path: pathlib.Path = pathlib.Path("./flatmates_csv_data.csv")
download_flatmates_listings(baseurl, csv_path)
# soup = BeautifulSoup(x, "html.parser")
# # soup.select("a[aria-label]")
# for i in range(1, 293):
#     data: dict[str, Any] = {"page": i}
#     print(f"Start:  https://flatmates.com.au/rooms/sydney/males+private-room?page={i}")
#     # x = requests.get("https://flatmates.com.au/share-houses/sydney/private-room?page=1")
#     listings = soup.find_all("a", {"class": "styles__contentBox___37_w9"})
#     for j, page_data in enumerate(listings, 1):
#         data["listing"] = j
#         df = pd.concat(df, parse_listing_tile(page_data) | data)
#     # pd.DataFrame([parse_listing_tile(tile) for tile in listings])
#     print(f"Stop:   https://flatmates.com.au/rooms/sydney/males+private-room?page={i}")

In [ ]:
dict(x.headers)["cache-control"]

In [ ]:
bs4.BeautifulSoup(client.get("https://flatmates.com.au/people/?page=1"))

In [70]:
import requests